In [4]:
from __future__ import division
import argparse
import torch
import matplotlib.pyplot as plt
from matplotlib.pyplot import imsave, imread
import getopt
import matplotlib
matplotlib.use('Agg')
import numpy as np
import math
import os
import PIL
import PIL.Image
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [5]:
#!/usr/bin/env python

#from video_processing import frame_capture


##########################################################

assert(int(str('').join(torch.__version__.split('.')[0:3])) >= 41) # requires at least pytorch version 0.4.1

torch.set_grad_enabled(False) # make sure to not compute gradients for computational performance

torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance

##########################################################

arguments_strModel = 'sintel-final'
arguments_strFirst = './images/first.png'
arguments_strSecond = './images/second.png'
arguments_strOut = './out.flo'

parser = argparse.ArgumentParser()
parser.add_argument('-gpuid', nargs=1, type=str, default='0')  # python3 main.py -gpuid=1,2,3
#args = parser.parse_args()
#os.environ['_VISIBLE_DEVICES'] = args.gpuid[0]
#print(os.environ['CUDA_VISIBLE_DEVICES'])
#
# for strOption, strArgument in getopt.getopt(sys.argv[1:], '', [ strParameter[2:] + '=' for strParameter in sys.argv[1::2] ])[0]:
# 	if strOption == '--model' and strArgument != '': arguments_strModel = strArgument # which model to use, see below
# 	if strOption == '--first' and strArgument != '': arguments_strFirst = strArgument # path to the first frame
# 	if strOption == '--second' and strArgument != '': arguments_strSecond = strArgument # path to the second frame
# 	if strOption == '--out' and strArgument != '': arguments_strOut = strArgument # path to where the output should be stored
# end

##########################################################

Backward_tensorGrid = {}

def Backward(tensorInput, tensorFlow):
	if str(tensorFlow.size()) not in Backward_tensorGrid:
		tensorHorizontal = torch.linspace(-1.0, 1.0, tensorFlow.size(3)).view(1, 1, 1, tensorFlow.size(3)).expand(tensorFlow.size(0), -1, tensorFlow.size(2), -1)
		tensorVertical = torch.linspace(-1.0, 1.0, tensorFlow.size(2)).view(1, 1, tensorFlow.size(2), 1).expand(tensorFlow.size(0), -1, -1, tensorFlow.size(3))

		Backward_tensorGrid[str(tensorFlow.size())] = torch.cat([ tensorHorizontal, tensorVertical ], 1).cuda()#
	# end

	tensorFlow = torch.cat([ tensorFlow[:, 0:1, :, :] / ((tensorInput.size(3) - 1.0) / 2.0), tensorFlow[:, 1:2, :, :] / ((tensorInput.size(2) - 1.0) / 2.0) ], 1)

	return torch.nn.functional.grid_sample(input=tensorInput, grid=(Backward_tensorGrid[str(tensorFlow.size())] + tensorFlow).permute(0, 2, 3, 1), mode='bilinear', padding_mode='border')
# end

##########################################################

class Network(torch.nn.Module):
	def __init__(self):
		super(Network, self).__init__()

		class Preprocess(torch.nn.Module):
			def __init__(self):
				super(Preprocess, self).__init__()
			# end

			def forward(self, tensorInput):
				tensorBlue = (tensorInput[:, 0:1, :, :] - 0.406) / 0.225
				tensorGreen = (tensorInput[:, 1:2, :, :] - 0.456) / 0.224
				tensorRed = (tensorInput[:, 2:3, :, :] - 0.485) / 0.229

				return torch.cat([ tensorRed, tensorGreen, tensorBlue ], 1)
			# end
		# end

		class Basic(torch.nn.Module):
			def __init__(self, intLevel):
				super(Basic, self).__init__()

				self.moduleBasic = torch.nn.Sequential(
					torch.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=7, stride=1, padding=3),
					torch.nn.ReLU(inplace=False),
					torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, stride=1, padding=3),
					torch.nn.ReLU(inplace=False),
					torch.nn.Conv2d(in_channels=64, out_channels=32, kernel_size=7, stride=1, padding=3),
					torch.nn.ReLU(inplace=False),
					torch.nn.Conv2d(in_channels=32, out_channels=16, kernel_size=7, stride=1, padding=3),
					torch.nn.ReLU(inplace=False),
					torch.nn.Conv2d(in_channels=16, out_channels=2, kernel_size=7, stride=1, padding=3)
				)
			# end

			def forward(self, tensorInput):
				return self.moduleBasic(tensorInput)
			# end
		# end

		self.modulePreprocess = Preprocess()

		self.moduleBasic = torch.nn.ModuleList([ Basic(intLevel) for intLevel in range(6) ])

		self.load_state_dict(torch.load('./network-' + arguments_strModel + '.pytorch'))
	# end

	def forward(self, tensorFirst, tensorSecond):
		tensorFirst = [ self.modulePreprocess(tensorFirst) ]
		tensorSecond = [ self.modulePreprocess(tensorSecond) ]

		for intLevel in range(5):
			if tensorFirst[0].size(2) > 32 or tensorFirst[0].size(3) > 32:
				tensorFirst.insert(0, torch.nn.functional.avg_pool2d(input=tensorFirst[0], kernel_size=2, stride=2, count_include_pad=False))
				tensorSecond.insert(0, torch.nn.functional.avg_pool2d(input=tensorSecond[0], kernel_size=2, stride=2, count_include_pad=False))
			# end
		# end

		tensorFlow = tensorFirst[0].new_zeros([ tensorFirst[0].size(0), 2, int(math.floor(tensorFirst[0].size(2) / 2.0)), int(math.floor(tensorFirst[0].size(3) / 2.0)) ])

		for intLevel in range(len(tensorFirst)):
			tensorUpsampled = torch.nn.functional.interpolate(input=tensorFlow, scale_factor=2, mode='bilinear', align_corners=True) * 2.0

			if tensorUpsampled.size(2) != tensorFirst[intLevel].size(2): tensorUpsampled = torch.nn.functional.pad(input=tensorUpsampled, pad=[ 0, 0, 0, 1 ], mode='replicate')
			if tensorUpsampled.size(3) != tensorFirst[intLevel].size(3): tensorUpsampled = torch.nn.functional.pad(input=tensorUpsampled, pad=[ 0, 1, 0, 0 ], mode='replicate')

			tensorFlow = self.moduleBasic[intLevel](torch.cat([ tensorFirst[intLevel], Backward(tensorInput=tensorSecond[intLevel], tensorFlow=tensorUpsampled), tensorUpsampled ], 1)) + tensorUpsampled
		# end

		return tensorFlow
	# end
# end

moduleNetwork = Network().cuda().eval()

##########################################################
def estimate(tensorFirst, tensorSecond):
	assert(tensorFirst.size(1) == tensorSecond.size(1))
	assert(tensorFirst.size(2) == tensorSecond.size(2))

	intWidth = tensorFirst.size(2)
	intHeight = tensorFirst.size(1)

#	assert(intWidth == 1024) # remember that there is no guarantee for correctness, comment this line out if you acknowledge this and want to continue
#	assert(intHeight == 416) # remember that there is no guarantee for correctness, comment this line out if you acknowledge this and want to continue

	tensorPreprocessedFirst = tensorFirst.cuda().view(1, 3, intHeight, intWidth)
	tensorPreprocessedSecond = tensorSecond.cuda().view(1, 3, intHeight, intWidth)

	intPreprocessedWidth = int(math.floor(math.ceil(intWidth / 32.0) * 32.0))
	intPreprocessedHeight = int(math.floor(math.ceil(intHeight / 32.0) * 32.0))

	tensorPreprocessedFirst = torch.nn.functional.interpolate(input=tensorPreprocessedFirst, size=(intPreprocessedHeight, intPreprocessedWidth), mode='bilinear', align_corners=False)
	tensorPreprocessedSecond = torch.nn.functional.interpolate(input=tensorPreprocessedSecond, size=(intPreprocessedHeight, intPreprocessedWidth), mode='bilinear', align_corners=False)

	tensorFlow = torch.nn.functional.interpolate(input=moduleNetwork(tensorPreprocessedFirst, tensorPreprocessedSecond), size=(intHeight, intWidth), mode='bilinear', align_corners=False)

	tensorFlow[:, 0, :, :] *= float(intWidth) / float(intPreprocessedWidth)
	tensorFlow[:, 1, :, :] *= float(intHeight) / float(intPreprocessedHeight)
	return tensorFlow[0, :, :, :].cpu()
# end

##########################################################
def make_color_wheel():
    """
    Generate color wheel according Middlebury color code
    :return: Color wheel
    """
    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR

    colorwheel = np.zeros([ncols, 3])

    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.transpose(np.floor(255*np.arange(0, RY) / RY))
    col += RY

    # YG
    colorwheel[col:col+YG, 0] = 255 - np.transpose(np.floor(255*np.arange(0, YG) / YG))
    colorwheel[col:col+YG, 1] = 255
    col += YG

    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.transpose(np.floor(255*np.arange(0, GC) / GC))
    col += GC

    # CB
    colorwheel[col:col+CB, 1] = 255 - np.transpose(np.floor(255*np.arange(0, CB) / CB))
    colorwheel[col:col+CB, 2] = 255
    col += CB

    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.transpose(np.floor(255*np.arange(0, BM) / BM))
    col += + BM

    # MR
    colorwheel[col:col+MR, 2] = 255 - np.transpose(np.floor(255 * np.arange(0, MR) / MR))
    colorwheel[col:col+MR, 0] = 255

    return colorwheel



def compute_color(u, v):
	"""
    compute optical flow color map
    :param u: optical flow horizontal map
    :param v: optical flow vertical map
    :return: optical flow in color code
    """
	[h, w] = u.shape
	img = np.zeros([h, w, 3])
	nanIdx = np.isnan(u) | np.isnan(v)
	u[nanIdx] = 0
	v[nanIdx] = 0

	colorwheel = make_color_wheel()
	ncols = np.size(colorwheel, 0)

	rad = np.sqrt(u**2+v**2)

	a = np.arctan2(-v, -u) / np.pi

	fk = (a+1) / 2 * (ncols - 1) + 1

	k0 = np.floor(fk).astype(int)

	k1 = k0 + 1
	k1[k1 == ncols+1] = 1
	f = fk - k0

	for i in range(0, np.size(colorwheel,1)):
		tmp = colorwheel[:, i]
		col0 = tmp[k0-1] / 255
		col1 = tmp[k1-1] / 255
		col = (1-f) * col0 + f * col1

		idx = rad <= 1
		col[idx] = 1-rad[idx]*(1-col[idx])
		notidx = np.logical_not(idx)

		col[notidx] *= 0.75
		img[:, :, i] = np.uint8(np.floor(255 * col*(1-nanIdx)))

	return img


In [ ]:
def calculate_and_output_optical_flows(input_dir, flow_output_dir, video_name):
    if not os.path.exists(flow_output_dir):
        os.mkdir(flow_output_dir)

    input_video_dir = f"{input_dir}/{video_name}"
    flow_output_video_dir = f"{flow_output_dir}/{video_name}"
    
    if not os.path.exists(flow_output_video_dir):
        os.mkdir(flow_output_video_dir)
    else:
        print("output folder already exists, not running analysis for file")
        return None
    
    num_frames = len(os.listdir(input_video_dir))
    if num_frames == 0:
        print(f"Unable to find any input files for video {video_name} in folder {input_video_dir}")
        
    for i in range(1, num_frames - 1):
        first_file = f"{input_video_dir}/{video_name}{i:05d}.jpeg"
        second_file = f"{input_video_dir}/{video_name}{(i+1):05d}.jpeg"

        first_image_np = np.array(PIL.Image.open(first_file)) 
        second_image_np = np.array(PIL.Image.open(second_file)) 

        tensorFirst = torch.FloatTensor(
        first_image_np[:, :, ::-1].transpose(2, 0, 1).astype(np.float32) * (1.0 / 255.0))
        tensorSecond = torch.FloatTensor(
        second_image_np[:, :, ::-1].transpose(2, 0, 1).astype(np.float32) * (1.0 / 255.0))

        tensorOutput = estimate(tensorFirst, tensorSecond)
        tensorOutput = tensorOutput.numpy()
        np.save(
            f"{flow_output_video_dir}/{video_name}{i:05d}",
            tensorOutput)

        if i % 100==0:
            print("finished comparing frame " + str(i) + " video " + video_name)

if __name__ == '__main__':
    input_dir = "videos/4-scaled-down-jpegs"
    flow_output_dir="videos/6-optical-flows-scaled-down"
    videos = os.listdir(input_dir)
    for video_name in videos:
        print(f"Processing video '{video_name}'")
        calculate_and_output_optical_flows(
            input_dir=input_dir,
            flow_output_dir=flow_output_dir,
            video_name=video_name)

Processing video 'ship'
output folder already exists, not running analysis for file
Processing video 'video_4'
output folder already exists, not running analysis for file
Processing video 'gardens'
output folder already exists, not running analysis for file
Processing video 'sharks'
output folder already exists, not running analysis for file
Processing video 'video_8'
output folder already exists, not running analysis for file
Processing video 'skyhouse'
output folder already exists, not running analysis for file
Processing video 'video_7'
output folder already exists, not running analysis for file
Processing video 'minecraft'
output folder already exists, not running analysis for file
Processing video 'snowplanet'
Processing video 'sculptures'
Processing video 'video_2'
finished comparing frame 100 video video_2
finished comparing frame 200 video video_2
finished comparing frame 300 video video_2
finished comparing frame 400 video video_2
finished comparing frame 500 video video_2
fin

## Calculate the sum of norms of the flows for each frame


In [40]:
def calc_sum_of_norms(flow_array):
    if shape(flow_array)[0] != 2:
        raise ValueError("Wrong input shape. Has to contain flow in two dimensions")
    squared = np.power(flow_array, 2)
    squared_sum = squared[0] + squared[1]
    norms = np.sqrt(squared_sum)
    return norms.sum()

def calc_flow_and_save(input_dir, output_dir, video_name):
    flows = []
    video_dir = f"{input_dir}/{video_name}"
    video_files = os.listdir(video_dir)
    video_files.sort()

    for video_file in video_files:
        x = np.load(f"{video_dir}/{video_file}")
        flows.append(calc_sum_of_norms(x))
    np.save(f"{output_dir}/{video_name}_flow", np.asarray(flows))

input_dir="videos/6-optical-flows-scaled-down"
output_dir="videos/7-final-aggregated-optical-flows-scaled-down"
videos = os.listdir(input_dir)
for video_name in videos:
    print(f"Processing video '{video_name}'")
    calc_flow_and_save(input_dir, output_dir, video_name)